In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [ ]:
custom_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=45),  # Adjusted rotation degrees
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=custom_transform,
)

train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=True)

val_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize the image
]),
)

val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(1024)

        self.fc1 = nn.Linear(1024 * 3 * 3, 2048)
        self.bn7 = nn.BatchNorm1d(2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.bn8 = nn.BatchNorm1d(1024)
        self.fc3 = nn.Linear(1024, 512)
        self.bn9 = nn.BatchNorm1d(512)
        self.fc4 = nn.Linear(512, 102)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn6(self.conv6(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.view(-1, 1024 * 3 * 3)

        x = self.dropout(F.relu(self.bn7(self.fc1(x))))
        x = self.dropout(F.relu(self.bn8(self.fc2(x))))
        x = F.relu(self.bn9(self.fc3(x)))
        x = self.fc4(x)

        return F.log_softmax(x, dim=1)


model = NeuralNetwork().to(device)

In [ ]:

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)


In [ ]:
# Training loop
def train_loop(dataloader, model, loss_fn, optimizer, device):
    model.train()
    running_loss = 0.0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# Validation loop
def val_loop(dataloader, model, loss_fn, device):
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= len(dataloader)
    correct /= len(dataloader.dataset)
    return val_loss, correct * 100


In [ ]:
# Training
epochs = 200
start = time.time()
best_val_loss = float('inf')
patience = 15
counter = 0

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer, device)
    val_loss, val_accuracy = val_loop(val_dataloader, model, loss_fn, device)

    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}")

    # Adjust learning rate based on validation loss
    scheduler.step(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping")
            break

end = time.time()
print(f"Training took: {(end - start) / 60:.2f} minutes!")

Epoch 1/200
Train Loss: 4.6339, Val Loss: 4.6289, Val Accuracy: 1.96
Epoch 2/200
Train Loss: 4.4227, Val Loss: 4.4695, Val Accuracy: 4.22
Epoch 3/200
Train Loss: 4.2485, Val Loss: 4.2230, Val Accuracy: 9.22
Epoch 4/200
Train Loss: 4.1005, Val Loss: 4.1022, Val Accuracy: 13.73
Epoch 5/200
Train Loss: 3.9708, Val Loss: 3.9720, Val Accuracy: 17.06
Epoch 6/200
Train Loss: 3.8278, Val Loss: 3.8607, Val Accuracy: 19.71
Epoch 7/200
Train Loss: 3.7071, Val Loss: 3.7645, Val Accuracy: 21.57
Epoch 8/200
Train Loss: 3.5853, Val Loss: 3.6875, Val Accuracy: 23.04
Epoch 9/200
Train Loss: 3.4804, Val Loss: 3.6019, Val Accuracy: 25.10
Epoch 10/200
Train Loss: 3.3600, Val Loss: 3.4818, Val Accuracy: 26.67
Epoch 11/200
Train Loss: 3.2537, Val Loss: 3.3864, Val Accuracy: 27.16
Epoch 12/200
Train Loss: 3.1685, Val Loss: 3.2914, Val Accuracy: 28.73
Epoch 13/200
Train Loss: 3.0552, Val Loss: 3.2623, Val Accuracy: 30.98
Epoch 14/200
Train Loss: 2.9519, Val Loss: 3.1759, Val Accuracy: 32.75
Epoch 15/200
Train